In [1]:
# OpenStreetMap Feature Extraction for Air Quality Analysis
# This script extracts nearby physical features (roads, industrial zones, dumps, agricultural fields)
# from OpenStreetMap using OSMnx library in small batches with error handling

import pandas as pd
import numpy as np
import osmnx as ox
import geopandas as gpd
from shapely.geometry import Point, box
import warnings
import time
from typing import Dict, List, Tuple
import sys

warnings.filterwarnings('ignore')

# ============================================================================
# CONFIGURATION
# ============================================================================

BATCH_SIZE = 5
SEARCH_RADIUS = 5000  # meters = 5KM
OUTPUT_FILE = "Air_Quality_with_OSM_Features_5KM.xlsx"
INPUT_FILE = "Air Quality Dataset 2021-2023.xlsx"

# ============================================================================
# FEATURE EXTRACTION FUNCTIONS
# ============================================================================

def extract_roads(lat: float, lon: float, radius: int) -> Dict:
    """Extract road features from OSM"""
    try:
        roads = ox.features_from_point(
            (lat, lon),
            tags={'highway': True},
            dist=radius
        )
        
        if len(roads) > 0:
            road_names = [roads.index.get_level_values(0).unique()[i] 
                         for i in range(min(5, len(roads.index.get_level_values(0).unique())))]
            return {
                'count': len(roads),
                'examples': ', '.join(str(r)[:40] for r in road_names),
                'status': 'success'
            }
        return {'count': 0, 'examples': '', 'status': 'success'}
    except Exception as e:
        return {'count': 0, 'examples': f"Error: {str(e)[:30]}", 'status': 'failed'}

def extract_industrial_zones(lat: float, lon: float, radius: int) -> Dict:
    """Extract industrial facilities from OSM"""
    try:
        tags = {'industrial': True, 'landuse': 'industrial'}
        industrial = ox.features_from_point((lat, lon), tags=tags, dist=radius)
        
        if len(industrial) > 0:
            ind_names = industrial.index.get_level_values(0).unique()[:5]
            return {
                'count': len(industrial),
                'examples': ', '.join(str(i)[:35] for i in ind_names),
                'status': 'success'
            }
        return {'count': 0, 'examples': '', 'status': 'success'}
    except Exception as e:
        return {'count': 0, 'examples': f"Error: {str(e)[:30]}", 'status': 'failed'}

def extract_dump_sites(lat: float, lon: float, radius: int) -> Dict:
    """Extract waste management and dump sites from OSM"""
    try:
        tags = {'waste': True}
        dumps = ox.features_from_point((lat, lon), tags=tags, dist=radius)
        
        if len(dumps) > 0:
            dump_names = dumps.index.get_level_values(0).unique()[:5]
            return {
                'count': len(dumps),
                'examples': ', '.join(str(d)[:35] for d in dump_names),
                'status': 'success'
            }
        return {'count': 0, 'examples': '', 'status': 'success'}
    except Exception as e:
        return {'count': 0, 'examples': f"Error: {str(e)[:30]}", 'status': 'failed'}

def extract_agricultural_fields(lat: float, lon: float, radius: int) -> Dict:
    """Extract agricultural areas from OSM"""
    try:
        tags = {'landuse': ['farmland', 'farm', 'agricultural', 'grass', 'meadow']}
        agriculture = ox.features_from_point((lat, lon), tags=tags, dist=radius)
        
        if len(agriculture) > 0:
            agr_names = agriculture.index.get_level_values(0).unique()[:5]
            return {
                'count': len(agriculture),
                'examples': ', '.join(str(a)[:35] for a in agr_names),
                'status': 'success'
            }
        return {'count': 0, 'examples': '', 'status': 'success'}
    except Exception as e:
        return {'count': 0, 'examples': f"Error: {str(e)[:30]}", 'status': 'failed'}

def extract_all_features(lat: float, lon: float, radius: int) -> Dict:
    """Extract all features for a location"""
    features = {
        'roads': extract_roads(lat, lon, radius),
        'industrial': extract_industrial_zones(lat, lon, radius),
        'dumps': extract_dump_sites(lat, lon, radius),
        'agriculture': extract_agricultural_fields(lat, lon, radius)
    }
    return features

# ============================================================================
# MAIN PROCESSING
# ============================================================================

def main():
    print(f"\n{'='*80}")
    print("OSM FEATURE EXTRACTION FOR AIR QUALITY ANALYSIS")
    print(f"{'='*80}\n")
    
    # Load dataset
    print(f"Loading dataset from {INPUT_FILE}...")
    df = pd.read_excel(INPUT_FILE, sheet_name=0)
    print(f"✓ Loaded {len(df)} records\n")
    
    # Get unique locations
    unique_locations = df[['Latitude', 'Longitude', 'Location', 'State / Union Territory',
                           'City / town / village']].drop_duplicates(
                           subset=['Latitude', 'Longitude']).reset_index(drop=True)
    
    print(f"Unique locations to process: {len(unique_locations)}")
    print(f"Processing parameters:")
    print(f"  - Batch size: {BATCH_SIZE}")
    print(f"  - Search radius: {SEARCH_RADIUS}m")
    print(f"\n{'='*80}\n")
    
    # Initialize results
    features_data = []
    total_batches = (len(unique_locations) - 1) // BATCH_SIZE + 1
    
    # Process in batches
    for batch_num in range(0, len(unique_locations), BATCH_SIZE):
        batch = unique_locations.iloc[batch_num:batch_num + BATCH_SIZE]
        batch_idx = (batch_num // BATCH_SIZE) + 1
        
        print(f"Batch {batch_idx}/{total_batches}:")
        print(f"Processing locations {batch_num + 1} to {min(batch_num + BATCH_SIZE, len(unique_locations))}\n")
        
        for idx, row in batch.iterrows():
            lat = row['Latitude']
            lon = row['Longitude']
            location_name = row['Location']
            city = row['City / town / village']
            state = row['State / Union Territory']
            
            try:
                # Extract features
                print(f"  Querying {city}, {state} ({lat:.4f}, {lon:.4f})...", end='', flush=True)
                features = extract_all_features(lat, lon, SEARCH_RADIUS)
                
                # Compile record
                record = {
                    'Location': location_name,
                    'City': city,
                    'State': state,
                    'Latitude': lat,
                    'Longitude': lon,
                    'Roads_count': features['roads']['count'],
                    'Roads_examples': features['roads']['examples'],
                    'Industrial_zones_count': features['industrial']['count'],
                    'Industrial_examples': features['industrial']['examples'],
                    'Dump_sites_count': features['dumps']['count'],
                    'Dump_examples': features['dumps']['examples'],
                    'Agricultural_fields_count': features['agriculture']['count'],
                    'Agricultural_examples': features['agriculture']['examples'],
                    'Query_status': 'Success'
                }
                
                features_data.append(record)
                
                print(f" ✓ Roads:{features['roads']['count']} | "
                      f"Industrial:{features['industrial']['count']} | "
                      f"Dumps:{features['dumps']['count']} | "
                      f"Agriculture:{features['agriculture']['count']}")
                
            except Exception as e:
                print(f" ✗ Error: {str(e)[:50]}")
                record = {
                    'Location': location_name,
                    'City': city,
                    'State': state,
                    'Latitude': lat,
                    'Longitude': lon,
                    'Roads_count': 0,
                    'Roads_examples': '',
                    'Industrial_zones_count': 0,
                    'Industrial_examples': '',
                    'Dump_sites_count': 0,
                    'Dump_examples': '',
                    'Agricultural_fields_count': 0,
                    'Agricultural_examples': '',
                    'Query_status': f'Failed: {str(e)[:40]}'
                }
                features_data.append(record)
            
            # Small delay between queries to avoid rate limiting
            time.sleep(0.5)
        
        print(f"\nBatch completed. Waiting before next batch...\n")
        time.sleep(2)
    
    # Create features dataframe
    features_df = pd.DataFrame(features_data)
    
    # Print summary statistics
    print(f"\n{'='*80}")
    print("EXTRACTION SUMMARY")
    print(f"{'='*80}\n")
    
    print(f"Total locations processed: {len(features_df)}")
    successful = (features_df['Query_status'] == 'Success').sum()
    print(f"Successful queries: {successful}/{len(features_df)}")
    
    print(f"\n{'Feature Statistics':^80}")
    print(f"{'-'*80}")
    print(f"{'Feature':<20} {'Total':<12} {'Mean':<12} {'Max':<12}")
    print(f"{'-'*80}")
    print(f"{'Roads':<20} {features_df['Roads_count'].sum():<12.0f} "
          f"{features_df['Roads_count'].mean():<12.1f} {features_df['Roads_count'].max():<12.0f}")
    print(f"{'Industrial':<20} {features_df['Industrial_zones_count'].sum():<12.0f} "
          f"{features_df['Industrial_zones_count'].mean():<12.1f} "
          f"{features_df['Industrial_zones_count'].max():<12.0f}")
    print(f"{'Dump Sites':<20} {features_df['Dump_sites_count'].sum():<12.0f} "
          f"{features_df['Dump_sites_count'].mean():<12.1f} {features_df['Dump_sites_count'].max():<12.0f}")
    print(f"{'Agriculture':<20} {features_df['Agricultural_fields_count'].sum():<12.0f} "
          f"{features_df['Agricultural_fields_count'].mean():<12.1f} "
          f"{features_df['Agricultural_fields_count'].max():<12.0f}")
    print(f"{'-'*80}\n")
    
    # Merge with original data
    print("Merging extracted features with original air quality data...")
    
    # Merge on latitude and longitude
    merged_df = df.merge(
        features_df[['Latitude', 'Longitude', 'Roads_count', 'Industrial_zones_count',
                     'Dump_sites_count', 'Agricultural_fields_count', 'Query_status']],
        on=['Latitude', 'Longitude'],
        how='left'
    )
    
    # Fill NaN values for unmatched records (should be minimal)
    merged_df['Roads_count'].fillna(0, inplace=True)
    merged_df['Industrial_zones_count'].fillna(0, inplace=True)
    merged_df['Dump_sites_count'].fillna(0, inplace=True)
    merged_df['Agricultural_fields_count'].fillna(0, inplace=True)
    merged_df['Query_status'].fillna('Not processed', inplace=True)
    
    # Create additional derived features
    merged_df['Urban_density_score'] = (merged_df['Roads_count'] / merged_df['Roads_count'].max()).round(2)
    merged_df['Industrial_presence'] = (merged_df['Industrial_zones_count'] > 0).astype(int)
    merged_df['Pollution_source_risk'] = (
        (merged_df['Industrial_zones_count'] * 0.4 + 
         merged_df['Dump_sites_count'] * 0.3 + 
         merged_df['Roads_count'] * 0.3) / 100
    ).round(2)
    merged_df['Green_area_ratio'] = (
        merged_df['Agricultural_fields_count'] / 
        (merged_df['Roads_count'] + merged_df['Agricultural_fields_count'])
    ).round(2)
    
    # Save to Excel
    print(f"\nSaving merged data to {OUTPUT_FILE}...")
    with pd.ExcelWriter(OUTPUT_FILE, engine='openpyxl') as writer:
        merged_df.to_excel(writer, sheet_name='All Data', index=False)
        features_df.to_excel(writer, sheet_name='OSM Features', index=False)
        
        # Summary statistics sheet
        summary_stats = pd.DataFrame({
            'Metric': ['Total Records', 'Unique Locations', 'Successful Queries',
                      'Mean Roads', 'Mean Industrial', 'Mean Dumps', 'Mean Agriculture',
                      'Max Roads', 'Max Industrial', 'Max Dumps', 'Max Agriculture'],
            'Value': [len(merged_df), len(features_df), successful,
                     f"{features_df['Roads_count'].mean():.1f}",
                     f"{features_df['Industrial_zones_count'].mean():.1f}",
                     f"{features_df['Dump_sites_count'].mean():.1f}",
                     f"{features_df['Agricultural_fields_count'].mean():.1f}",
                     features_df['Roads_count'].max(),
                     features_df['Industrial_zones_count'].max(),
                     features_df['Dump_sites_count'].max(),
                     features_df['Agricultural_fields_count'].max()]
        })
        summary_stats.to_excel(writer, sheet_name='Summary', index=False)
    
    print(f"✓ Successfully saved to {OUTPUT_FILE}\n")
    
    # Display sample results
    print(f"{'Sample Results (First 15 locations):':^80}")
    print(merged_df[['City / town / village', 'State / Union Territory', 'Roads_count',
                     'Industrial_zones_count', 'Dump_sites_count', 'Agricultural_fields_count',
                     'Pollution_source_risk', 'Green_area_ratio']].head(15))
    
    print(f"\n{'='*80}")
    print("✓ Processing Complete!")
    print(f"{'='*80}\n")
    
    return merged_df, features_df

if __name__ == "__main__":
    merged_df, features_df = main()


OSM FEATURE EXTRACTION FOR AIR QUALITY ANALYSIS

Loading dataset from Air Quality Dataset 2021-2023.xlsx...
✓ Loaded 1068 records

Unique locations to process: 544
Processing parameters:
  - Batch size: 5
  - Search radius: 5000m


Batch 1/109:
Processing locations 1 to 5

  Querying Amaravati, Andhra Pradesh (16.4942, 80.5106)... ✓ Roads:2332 | Industrial:0 | Dumps:0 | Agriculture:89
  Querying Anatapur, Andhra Pradesh (14.6824, 77.6017)... ✓ Roads:5915 | Industrial:3 | Dumps:0 | Agriculture:2
  Querying Chittor, Andhra Pradesh (13.2162, 79.1051)... ✓ Roads:1986 | Industrial:4 | Dumps:0 | Agriculture:9
  Querying Eluru, Andhra Pradesh (16.7104, 81.1154)... ✓ Roads:4515 | Industrial:2 | Dumps:0 | Agriculture:1
  Querying Guntur, Andhra Pradesh (16.2915, 80.4542)... ✓ Roads:7442 | Industrial:4 | Dumps:0 | Agriculture:37

Batch completed. Waiting before next batch...

Batch 2/109:
Processing locations 6 to 10

  Querying Kadapa, Andhra Pradesh (14.4753, 78.8217)... ✓ Roads:3546 | Indust